In [13]:
print("Here we go again, hello world")

Here we go again, hello world


In [14]:
from bs4 import BeautifulSoup as bs
import requests
import pandas
from datetime import date

In [19]:
base_url = 'https://winestyle.ru'

def parse_product(link, cookies):

    global error_occured
    
    product_data = {}
    
    #запрос
    product_url = base_url+link
    product_data['url'] = product_url
    headers = {
        'Host': 'winestyle.ru',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.3.1 Safari/605.1.15',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'ru',
        'Accept-Encoding': 'gzip, deflate',
        # 'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        # 'Sec-Fetch-User': '?1',
        'Priority': 'u=0, i',
        'Cookie': cookies
    }

    print(product_url)
    # print(cookies)

    request = requests.get(product_url, headers=headers)
    code = request.status_code
    print(code)
    if not code == 200:
        error_occured = True
        return {},{}
    
    # парсинг 
    soup = bs(request.text, "html.parser")
    
    # код товара
    try:
        product_code = soup.find('span', class_='sku').text
        product_data['product_code'] = product_code
    except:
        print('no product code')
    
    # название
    try:
        title = soup.find('h2', class_='heading heading--4xl').parent.find('p').text
        product_data['tltle'] = title
    except:
        print('no title')
    
    # цена со скидкой и без - разные структуры 
    price_data = soup.find('div', class_='m-productpage-price')
    lg = price_data.find('p', class_='price price--lg')
    if lg is not None:
        product_data['price_main'] = lg.text
    else:
        price = price_data.find('div', class_='price price--sale')
        if price is not None:
            product_data['price_discount'] = price.contents[0].text
            product_data['price_main'] = price.contents[1].text

    #наличие
    try:
        stock = soup.find('span', class_='m-productpage-price__status').text
        product_data['stock'] = stock
    except:
        print('no stock info')
        
    # атрибуты
    wine_attributes = soup.find_all('li', class_='m-params dot')
    for attribute in wine_attributes:
        attribute_name = attribute.contents[0].text
        attribute_values = attribute.find_all('meta', itemprop="value")
        values_string = ''
        for value in attribute_values:
            values_string = values_string +'; '+ value['content']
        product_data[attribute_name] = values_string[2:]
        
    return product_data, request.cookies
    
    
def parse_page(link, number, cookie):

    global error_occured
    
    page_url = base_url + link 
    if number > 1:
        page_url = base_url + link + "?page=" + str(number)
    headers = {
        'Host': 'winestyle.ru',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.3.1 Safari/605.1.15',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'ru',
        'Accept-Encoding': 'gzip, deflate',
        # 'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        # 'Sec-Fetch-User': '?1',
        'Priority': 'u=0, i',
        'Cookie': cookie
    }

    print(page_url)
    # print(cookie)
    # print(headers)
    request = requests.get(page_url, headers=headers)
    code = request.status_code
    print(code)
    if not code == 200:
        error_occured = True
        return 
    
    # суп
    soup = bs(request.text, "html.parser")
    products = soup.find_all('p', class_='m-catalog-item__link')
    product_links = []
    for product in products:
        product_links.append(product.parent['href'])
    new_product_links = list(set(product_links))
    return new_product_links, request.cookies

In [20]:

def manage_cookies(cookies):
    # обход защиты
    global fresh_token
    global fresh_qrator
    global fresh_session
    
    # print(cookies)
    try:
        newCookieToken = cookies['CSRF-Token'] 
        # print("new token: " + str(newCookieToken))
        fresh_token = newCookieToken
    except:
        pass 
    try:
        new_qrator = cookies['qrator_msid2']
        print('new qrator_msid2: '+ str(new_qrator))
        fresh_qrator = new_qrator
    except:
        pass 
    try:
        new_session_token = cookies['SHOP_SESSION_TOKEN']
        print('new SHOP_SESSION_TOKEN: '+ str(new_qrator_jsr))
        fresh_session = new_session_token
    except:
        pass 
    return

def parse_prodcuts_page(page_link, page_number):
    products = []
    
    product_links, cookies = parse_page(page_link,page_number,build_cookies(fresh_token, fresh_qrator))
    
    manage_cookies(cookies)

    stock = ''
    product_type = ''
    
    match page_link:
        case '/wine/russia/available_ll/':
            stock = 'in_stock'
            product_type = 'wine'
        case '/champagnes-and-sparkling/russia/available_ll/':
            stock = 'in_stock'
            product_type = 'champagnes-and-sparkling'
        case _:
            pass 
    
    print("найдено товаров на странице " + str(i) + ": " +str(len(product_links)))
    
    for link in product_links:
        product, cookies = parse_product(link, build_cookies(fresh_token, fresh_qrator))
        manage_cookies(cookies)
        # проверка на сбой
        if not cookies:
            break
        product["product_type"] = product_type
        product["stock"] = stock
        products.append(product)
    return products
    
# Сначала парсим все, что в наличии


In [21]:
fresh_token = 'e3e51142-6849-4f8e-a417-bf0aa471ddd3'
fresh_qrator = 'v2.0.1744656622.976.6dfc9dc52LxpuWHD|TWK2aqbqERmp2ApF|ysMyekkiNDOwM4RLr9qMPtBQXx/Akq1sXj5Sw8uR+kNC3Z5rPgTOHP0JEqDpZquUsQupKjn2DSn2aPMvmD9buCimNjtRpGVozhzFR4d7JcU=-AO71X3udsd6ovjpNN8278hY8SyI='
fresh_session = 'eh4mpoid9j5sbs4f03lqvden3sm8gi7u'
error_occured = False 
# ' + str(token) + '
# ' + str(qrator) + '

def build_cookies(token, qrator):
    return 'customer=%7B%22firstName%22%3A%22%D0%90%D1%80%D1%82%D0%B5%D0%BC%22%2C%22lastName%22%3A%22%D0%9F%D0%B0%D0%B2%D0%BB%D0%BE%D0%B2%22%2C%22phone%22%3A%22%2B79166184948%22%2C%22email%22%3A%22art_pavlov00%40mail.ru%22%2C%22onPointLoyalty%22%3Afalse%2C%22discountCard%22%3A%22%22%7D; customer-is-verified=true; ibwsc=599718; _ym_isad=2; _ym_d=1740743012; _ym_uid=1740743012155285109; redirected-to-from=1; LastOrderDate=1737813515; e_mail=apple_dev.arty%40mail.ru; CheckoutVer=new_checkout'+fresh_token+'; SHOP_SESSION_TOKEN='+fresh_session+'; customer-is-verified=false; qrator_msid2='+fresh_qrator


products_data = []

# in stock russia, по 36 товаров на странице
page_from_1 = 1
page_to_1 = 59
page_from_2 = 1
page_to_2 = 13

# # all russia 
# page_from_1 = 1
# page_to_1 = 243
# page_from_2 = 1
# page_to_2 = 39

# Тихие вина 
for i in range(page_from_1, page_to_1):

    if error_occured: break
    
    still_in_stock = parse_prodcuts_page("/wine/russia/available_ll/",i)
    # still_in_stock = parse_prodcuts_page("/wine/russia/",i)
    
    # # проверка на сбой
    # if not still_in_stock: break

    products_data.extend(still_in_stock)

# Игристые вина 
for i in range(page_from_2,page_to_2):
    
    if error_occured: break
    
    sparkling_in_stock = parse_prodcuts_page("/champagnes-and-sparkling/russia/available_ll/",i)
    # sparkling_in_stock = parse_prodcuts_page("/champagnes-and-sparkling/russia/",i)
    
    # # проверка на сбой
    # if not sparkling_in_stock: break

    products_data.extend(sparkling_in_stock)
        
print('получена инфорация о товарах:' + str(len(products_data)))

    
df = pandas.DataFrame(data=products_data)
today = date.today()
df.to_csv("WineStyle_stock_"+str(page_from_1)+'-'+str(page_to_1)+'-'+str(page_from_2)+'-'+str(page_to_2)+'_'+str(today)+".csv")
# df.to_csv("WineStyle_all_"+str(page_from_1)+'-'+str(page_to_1)+'-'+str(page_from_2)+'-'+str(page_to_2)+'_'+str(today)+".csv")

https://winestyle.ru/wine/russia/available_ll/
200
найдено товаров на странице 1: 36
https://winestyle.ru/products/Zolotoe-Pole-Kaffa-Saperavi.html
200
https://winestyle.ru/products/South-Vertical-Cabernet-Franc-2022.html
200
https://winestyle.ru/products/Zolotaya-Balka-ZB-Wine-Riesling.html
200
https://winestyle.ru/products/Semisam-Red-Dry-2022.html
200
https://winestyle.ru/products/Zolotaya-Balka-ZB-Wine-Rose-Dry.html
200
https://winestyle.ru/products/Uroki-Francuzskogo-Krasnostop.html
200
https://winestyle.ru/products/Alma-Valley-Pique-Nique-White-Semi-Dry-2023.html
200
https://winestyle.ru/products/Uroki-Francuzskogo-Cabernet-Franc.html
200
https://winestyle.ru/products/Myskhako-Quintessence-Zinfandel.html
200
https://winestyle.ru/products/Malbec-de-Gai-Kodzor-2023.html
200
https://winestyle.ru/products/Golubitskoe-Estate-Merlot-2022.html
200
https://winestyle.ru/products/Zolotoe-Pole-Kaffa-Cabernet-Franc.html
200
https://winestyle.ru/products/Agora-Riesling.html
200
https://winest